<a href="https://colab.research.google.com/github/yasindusamarawickrama/OHTS/blob/main/Port_Scanner%20(Machine%20Learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

df = pd.read_csv('/content/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')



In [ ]:
df.drop(columns = [' Fwd Header Length'])
df.fillna(0)

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.12-192.168.10.50-35396-22-6,192.168.10.12,35396,192.168.10.50,22,6,7/7/2017 1:00,1266342,41,44,...,32,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.16-192.168.10.50-60058-22-6,192.168.10.16,60058,192.168.10.50,22,6,7/7/2017 1:00,1319353,41,44,...,32,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.12-192.168.10.50-35396-22-6,192.168.10.12,35396,192.168.10.50,22,6,7/7/2017 1:00,160,1,1,...,32,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.12-192.168.10.50-35398-22-6,192.168.10.12,35398,192.168.10.50,22,6,7/7/2017 1:00,1303488,41,42,...,32,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.12-192.168.10.50-35396-22-6,192.168.10.50,22,192.168.10.12,35396,6,7/7/2017 1:00,77,1,2,...,32,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2613,192.168.10.1-192.168.10.3-53-61793-17,192.168.10.3,61793,192.168.10.1,53,17,7/7/2017 1:08,31122,1,1,...,32,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,BENIGN
2614,192.168.10.1-192.168.10.3-53-61422-17,192.168.10.3,61422,192.168.10.1,53,17,7/7/2017 1:08,101680,1,1,...,32,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,BENIGN
2615,192.168.10.3-192.168.10.16-53-22529-17,192.168.10.16,22529,192.168.10.3,53,17,7/7/2017 1:08,31936,2,2,...,32,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,BENIGN
2616,192.168.10.3-192.168.10.16-53-32926-17,192.168.10.16,32926,192.168.10.3,53,17,7/7/2017 1:08,221,2,2,...,32,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,BENIGN


In [ ]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
df['Flow ID']= label_encoder.fit_transform(df['Flow ID'])
df['Flow ID'].unique()

label_encoder = preprocessing.LabelEncoder()
df[' Timestamp']= label_encoder.fit_transform(df[' Timestamp'])
df[' Timestamp'].unique()

label_encoder = preprocessing.LabelEncoder()
df[' Source IP']= label_encoder.fit_transform(df[' Source IP'])
df[' Source IP'].unique()

label_encoder = preprocessing.LabelEncoder()
df[' Destination IP']= label_encoder.fit_transform(df[' Destination IP'])
df[' Destination IP'].unique()

array([1125, 1115, 1118, ..., 1937,  420,  277])

In [ ]:
from sklearn.preprocessing import LabelEncoder

def Encoder(df):
          columnsToEncode = list(df.select_dtypes(include=['category','object']))
          le = LabelEncoder()
          for feature in columnsToEncode:
              try:
                  df[feature] = le.fit_transform(df[feature])
              except:
                  print('Error encoding '+feature)
          return df

In [ ]:
df = Encoder(df)

In [ ]:
features = ['Total Length of Fwd Packets',' Fwd Packet Length Max',' Flow IAT Mean',' Total Length of Bwd Packets']
X = df.loc[:, features]
y = df.loc[:, [' Label']]

In [ ]:
X

,Total Length of Fwd Packets,Fwd Packet Length Max,Flow IAT Mean,Total Length of Bwd Packets
0,2664,456,15075.500000,6954
1,2664,456,15706.583330,6954
2,0,0,160.000000,0
3,2728,456,15896.195120,6634
4,0,0,38.500000,0
...,...,...,...,...
2613,64,64,31122.000000,157
2614,46,46,101680.000000,253
2615,76,38,10645.333330,140
2616,76,38,73.666667,196


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = 0.2)

In [ ]:
y_test

,Label
1160,0
667,0
2118,0
1462,0
1836,0
...,...
46,0
747,0
2397,0
753,0


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn5 = KNeighborsClassifier(n_neighbors = 5)
knn1 = KNeighborsClassifier(n_neighbors=1)

In [ ]:
knn5.fit(X_train, y_train)
knn1.fit(X_train, y_train)

y_pred_5 = knn5.predict(X_test)
y_pred_1 = knn1.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
from sklearn.metrics import accuracy_score

print("Accuracy with k=5", accuracy_score(y_test, y_pred_5)*100)
print("Accuracy with k=1", accuracy_score(y_test, y_pred_1)*100)

Accuracy with k=5 99.71360381861575
Accuracy with k=1 99.0453460620525


In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

<ipython-input-70-efede872133b>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


RandomForestClassifier()

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100)

Accuracy: 99.79578835295453


In [ ]:
df = pd.read_csv('/content/Test.csv')

In [ ]:
features = ['Length']
X = df.loc[:, features]


In [ ]:
y_pred = rf.predict(X)

ValueError: ignored

In [ ]:
print (y_pred)

[0 0 0 ... 0 0 0]


In [ ]:
for prediction in y_pred:
    print(prediction)

Streaming output truncated to the last 5000 lines.
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

In [ ]:
import joblib

# save model with joblib
filename = 'port_scanner.sav'
joblib.dump(knn5, filename)

['port_scanner.sav']

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# load model with joblib
loaded_model = joblib.load(filename)

# evaluate model
y_predict = knn5.predict(X_test)

# check results
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2089
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         1

    accuracy                           1.00      2095
   macro avg       0.33      0.33      0.33      2095
weighted avg       0.99      1.00      1.00      2095



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import joblib

# save model with joblib
filename = 'port_scanner.sav'
joblib.dump(knn1, filename)

['port_scanner.sav']

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# load model with joblib
loaded_model = joblib.load(filename)

# evaluate model
y_predict = knn1.predict(X_test)

# check results
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      2089
           1       0.11      0.40      0.17         5
           2       0.00      0.00      0.00         1

    accuracy                           0.99      2095
   macro avg       0.37      0.46      0.39      2095
weighted avg       1.00      0.99      0.99      2095



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = knn1
model = joblib.load("/content/port_scanner.sav")

input_data = [[2664, 456, 15075.500000, 6954], [0, 0, 38.500000, 0], [46, 46 ,101680.000000, 253]]

predictions = model.predict(input_data)
print(predictions)

[0 0 0]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
